In [1]:
#download repo
!git clone https://github.com/victorchall/EveryDream.git
# Set working directory
%cd EveryDream

c:\Users\texas\Desktop\School\SceneFusion\AnimateDiff-SceneFusion\data\EveryDream


Cloning into 'EveryDream'...


In [2]:

# install requirements
# !pip install torch=='1.12.1+cu113' 'torchvision==0.13.1+cu113' --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pandas>='1.3.5'
!git clone https://github.com/salesforce/BLIP scripts/BLIP
!pip install timm
!pip install fairscale
# !pip install transformers=='4.19.2'
!pip install timm
!pip install aiofiles
!pip install aiohttp


Cloning into 'scripts/BLIP'...


     ---------------------------------------- 0.0/59.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.8 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.8 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 59.8/59.8 kB 533.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.2 MB 2.3 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/2.2 MB 4.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.2 MB 4.9 MB/s eta 0:00:01
   -------------------------- ------------- 1.5/2.2 MB 8.5 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/266.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/266.3 kB ? eta -:--:--
     ---- -------------

In [5]:
# !pip uninstall pillow
# !pip install pillow

^C


In [ ]:
%cd ../data

In [9]:
# !python EveryDream/scripts/auto_caption.py --img_dir raw/1 --out_dir output 

c:\Users\texas\Desktop\School\SceneFusion\AnimateDiff-SceneFusion\data
Windows detected, using asyncio.WindowsSelectorEventLoopPolicy
BLIP not found, cloning BLIP repo
starting
input_dir:  raw/1
Model cached to: .cache/model_base_caption_capfilt_large.pth
load checkpoint from .cache/model_base_caption_capfilt_large.pth
loading model to cuda
working image:  raw/1\Screenshot 2023-11-12 155626.png
working image:  raw/1\Screenshot 2023-11-12 155646.png
working image:  raw/1\Screenshot 2023-11-12 155700.png
working image:  raw/1\Screenshot 2023-11-12 155712.png
working image:  raw/1\Screenshot 2023-11-12 155727.png
working image:  raw/1\Screenshot 2023-11-12 155746.png


Cloning into 'scripts/BLIP'...


In [10]:
from torchvision import transforms
SIZE = 256

c:\Users\texas\anaconda3\envs\animatediff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def load_image(raw_image, device):
    transform = transforms.Compose([
        #transforms.CenterCrop(SIZE),
        transforms.Resize((SIZE, SIZE), interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    image = transform(raw_image).unsqueeze(0).to(device)
    return image

In [31]:
import os
import sys
import torch
import aiofiles
import io
sys.path.append("EveryDream/scripts/BLIP")

import models.blip

config_path = "EveryDream/scripts/BLIP/configs/med_config.json"

cache_folder = ".cache"
model_cache_path = ".cache/model_base_caption_capfilt_large.pth"

blip_decoder = models.blip.blip_decoder(pretrained=model_cache_path, image_size=SIZE, vit='base', med_config=config_path)
blip_decoder.eval()

print(f"loading model to cuda")

blip_decoder = blip_decoder.to(torch.device("cuda"))
    

reshape position embedding from 576 to 256
load checkpoint from .cache/model_base_caption_capfilt_large.pth
loading model to cuda


In [62]:
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
import cv2

count = os.listdir(f'raw')
print(count)
minFileCount = 1000
folder = 1

# create csv file
import pandas as pd
df = pd.DataFrame(columns=['videoid', 'name', 'page_dir'])

for i in range(1, len(count)+1):
    # get list of images from raw/{i} folder
    images = os.listdir(f'raw/{i}')
    
    if len(images) < minFileCount:
        minFileCount = len(images)
        folder = i
    
    # rename images to 1.jpg, 2.jpg, 3.jpg, etc
    for j in range(len(images)):
        os.rename(f'raw/{i}/{images[j]}', f'raw/{i}/{j}.png')
        images[j] = f'raw/{i}/{j}.png'
    
    img_file_name = images[0]
    
    async with aiofiles.open(img_file_name, "rb") as input_file:
        print("working image: ", img_file_name)

        image_bin = await input_file.read()
        image = Image.open(io.BytesIO(image_bin))

        if not image.mode == "RGB":
            image = image.convert("RGB")

        image = load_image(image, device=torch.device("cuda"))
        # caption the image
        captions = blip_decoder.generate(image, sample=False, num_beams=16, min_length=22, max_length=48, repetition_penalty=1.0)
        caption = captions[0]
        
        # save caption as name to csv file
        df.loc[len(df)] = [i, caption, f'raw/{i}']
        
        # print(caption)
        
        # turn images into video
        # Get the images
        images = [cv2.imread(image_path) for image_path in images]
        # crop image to center square shape using the shortest side
        images = [image[:, int((image.shape[1] - image.shape[0]) / 2):int((image.shape[1] + image.shape[0]) / 2)] for image in images]
        # resize images to 256x256
        images = [cv2.resize(image, (256, 256)) for image in images]

        # Create a video writer
        video_writer = cv2.VideoWriter(f'output/{i}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (images[0].shape[1], images[0].shape[0]))

        # Write the images to the video writer
        for image in images:
            video_writer.write(image)
            
print("minFileCount: ", minFileCount)
            
# save csv file
df.to_csv('output.csv', index=False)

['1', '2', '3']
working image:  raw/1/0.png
working image:  raw/2/0.png
working image:  raw/3/0.png
minFileCount:  5
